## Show the tables in schema

In [13]:
import sys
import os
sys.path.append(os.path.abspath(".."))  # adding the parent directory of 'notebooks' to sys.path
from db_utils.db_setup import Database
from sqlalchemy import text
engine = Database()
schema_list = ["mimiciv_icu", "mimiciv_hosp"]
for schema in schema_list:
    print(f"Schema: {schema}")
    tables = Database.show_tables_in_schema(engine, schema)
    print(tables)

2025-06-13 14:23:13,287 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-13 14:23:13,287 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-13 14:23:13,288 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-13 14:23:13,288 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-13 14:23:13,289 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-13 14:23:13,289 INFO sqlalchemy.engine.Engine [raw sql] {}
Schema: mimiciv_icu
2025-06-13 14:23:13,291 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-13 14:23:13,291 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema = %(schema)s
2025-06-13 14:23:13,291 INFO sqlalchemy.engine.Engine [generated in 0.00060s] {'schema': 'mimiciv_icu'}
['caregiver', 'chartevents', 'datetimeevents', 'd_items', 'icustays', 'ingredientevents', 'inputevents', 'outputevents', 'procedureevents']
Schema: mimiciv_hosp
2025-06-13 14:23:13,295 INFO sqlalchemy.engine.Engine SELECT 

# Getting all `labevents` data and filtering

##### Fetching `demographic` data

In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine, text
from psycopg2.extras import execute_values

# Load environment variables
load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

# Create SQLAlchemy engine
url = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(url)
conn = engine.connect()
cursor = conn.connection.cursor()

# Creating a TEMPORARY table
cursor.execute("""
    CREATE TEMP TABLE temp_cohort (
        subject_id INT,
        hadm_id INT,
        admittime TIMESTAMP,
        dischtime TIMESTAMP,
        target  INT
    );
""")

# Loading the CSV and insert into temp_cohort
cohort_df = pd.read_csv('../assets/cohort1_target.csv')
cohort_df['admittime'] = pd.to_datetime(cohort_df['admittime'], errors='coerce')
cohort_df['dischtime'] = pd.to_datetime(cohort_df['dischtime'], errors='coerce')

values = list(cohort_df.itertuples(index=False, name=None))
execute_values(cursor,
    "INSERT INTO temp_cohort (subject_id, hadm_id, admittime, dischtime, target) VALUES %s",
    values
)

# Fetching demographic data from admissions table
cursor.execute("""
    SELECT 
        c.subject_id,
        c.hadm_id,
        c.admittime,
        c.dischtime,
        c.target,
        p.gender,
        p.anchor_age,
        a.race
    FROM temp_cohort c
    JOIN mimiciv_hosp.admissions a ON c.hadm_id = a.hadm_id
    JOIN mimiciv_hosp.patients p ON a.subject_id = p.subject_id

""")

rows = cursor.fetchall()

# Finally creating DataFrame 
columns = ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'target', 'gender', 'anchor_age', 'race']
final_df = pd.DataFrame(rows, columns=columns)
# Save the final DataFrame to a Parquet file
final_df['admittime'] = pd.to_datetime(final_df['admittime'], errors='coerce')
final_df['dischtime'] = pd.to_datetime(final_df['dischtime'], errors='coerce')
final_df['anchor_age'] = pd.to_numeric(final_df['anchor_age'], errors='coerce')
final_df['target'] = pd.to_numeric(final_df['target'], errors='coerce')
final_df.to_parquet("../dataset/raw/cohort_with_demographic_data.parquet", index=False)

# Finalize
conn.connection.commit()
cursor.close()
conn.close()

In [4]:
demog_df = pd.read_parquet("../dataset/raw/cohort_with_demographic_data.parquet")
demog_df.head(5)  # Display the first 5 rows of the raw Parquet file

,subject_id,hadm_id,admittime,dischtime,target,gender,anchor_age,race
0,10010231,23835132,2118-04-02 11:54:00,2118-04-07 11:26:00,0,M,57,HISPANIC/LATINO - GUATEMALAN
1,10010231,23835132,2118-04-02 11:54:00,2118-04-07 11:26:00,0,M,57,HISPANIC/LATINO - GUATEMALAN
2,10010231,23835132,2118-04-02 11:54:00,2118-04-07 11:26:00,0,M,57,HISPANIC/LATINO - GUATEMALAN
3,10010231,24995642,2118-02-21 13:30:00,2118-02-26 16:50:00,0,M,57,HISPANIC/LATINO - GUATEMALAN
4,10010231,24995642,2118-02-21 13:30:00,2118-02-26 16:50:00,0,M,57,HISPANIC/LATINO - GUATEMALAN


In [14]:
demog_df['gender'].unique()  # Check unique values in the

array(['M', 'F'], dtype=object)

In [5]:
demog_df['anchor_age'].max(), demog_df['anchor_age'].min()

(np.int64(91), np.int64(18))

In [6]:
from sklearn.preprocessing import LabelEncoder
def map_race(race):
    if pd.isna(race):
        return 'Unknown or Not Reported'
    
    race = race.upper()
    
    if 'HISPANIC' in race or 'LATINO' in race or 'SOUTH AMERICAN' in race:
        return 'Hispanic or Latino'
    elif 'WHITE' in race:
        return 'White'
    elif 'BLACK' in race or 'AFRICAN' in race:
        return 'Black or African American'
    elif 'ASIAN' in race:
        return 'Asian'
    elif 'PACIFIC ISLANDER' in race or 'NATIVE HAWAIIAN' in race:
        return 'Native Hawaiian or Other Pacific Islander'
    elif 'AMERICAN INDIAN' in race or 'ALASKA NATIVE' in race:
        return 'American Indian or Alaska Native'
    elif 'DECLINED' in race or 'UNABLE' in race or 'UNKNOWN' in race:
        return 'Unknown or Not Reported'
    else:
        return 'Other'

le = LabelEncoder()
demog_df['race_grouped'] = demog_df['race'].apply(map_race)  # apply your earlier grouping
demog_df['race_target'] = le.fit_transform(demog_df['race_grouped'])


In [6]:
# unique ages and races
print("Unique ages:", final_df['anchor_age'].unique())
print("Unique races:", final_df['race'].unique())

Unique ages: [57 58 60 72 59 73 75 74 41 61 65 45 71 78 24 50 77 63 69 91 44 42 76 84
 56 67 55 80 46 68 47 32 53 33 52 48 30 85 66 83 87 64 81 36 26 79 28 43
 70 27 62 25 49 54 89 21 20 82 34 51 40 29 31 86 38 23 88 39 35 22 37 18
 19]
Unique races: ['HISPANIC/LATINO - GUATEMALAN' 'WHITE' 'BLACK/AFRICAN AMERICAN' 'OTHER'
 'ASIAN - CHINESE' 'ASIAN - SOUTH EAST ASIAN' 'ASIAN' 'UNKNOWN'
 'WHITE - OTHER EUROPEAN' 'UNABLE TO OBTAIN' 'PATIENT DECLINED TO ANSWER'
 'WHITE - RUSSIAN' 'SOUTH AMERICAN' 'WHITE - BRAZILIAN'
 'HISPANIC/LATINO - DOMINICAN' 'BLACK/AFRICAN' 'PORTUGUESE'
 'HISPANIC/LATINO - PUERTO RICAN' 'BLACK/CAPE VERDEAN'
 'HISPANIC/LATINO - HONDURAN' 'HISPANIC/LATINO - CENTRAL AMERICAN'
 'BLACK/CARIBBEAN ISLAND' 'ASIAN - ASIAN INDIAN'
 'WHITE - EASTERN EUROPEAN' 'HISPANIC/LATINO - COLUMBIAN'
 'HISPANIC/LATINO - SALVADORAN' 'HISPANIC/LATINO - CUBAN'
 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' 'ASIAN - KOREAN'
 'HISPANIC/LATINO - MEXICAN' 'AMERICAN INDIAN/ALASKA NATIVE']


##### Fetching labevents data prior `7` or `14` days

In [8]:
raw_parquet = pd.read_parquet("../dataset/raw/final_lab_events_7_days.parquet")
raw_parquet.head(5)  # Display the first 5 rows of the raw Parquet file

,subject_id,hadm_id,itemid,charttime,valuenum,dischtime,target,gender,anchor_age,race
0,10010231,29368887,51233,2118-01-15 17:45:00,NaN,2118-01-20 14:00:00,1,M,57,HISPANIC/LATINO - GUATEMALAN
1,10010231,29368887,51233,2118-01-15 17:45:00,NaN,2118-01-20 14:00:00,1,M,57,HISPANIC/LATINO - GUATEMALAN
2,10010231,29368887,51233,2118-01-15 17:45:00,NaN,2118-01-20 14:00:00,1,M,57,HISPANIC/LATINO - GUATEMALAN
3,10010231,21586397,51678,2117-12-19 06:20:00,6.0,2117-12-23 16:51:00,1,M,57,HISPANIC/LATINO - GUATEMALAN
4,10010231,21586397,51678,2117-12-19 06:20:00,6.0,2117-12-23 16:51:00,1,M,57,HISPANIC/LATINO - GUATEMALAN


In [9]:
len(raw_parquet)  # Display the number of rows in the raw Parquet file


10537677

In [ ]:
# First get all unique patient IDs
patient_ids = pd.read_sql("SELECT DISTINCT subject_id FROM public.temp_cohort ORDER BY subject_id", engine)

lab_df = pd.DataFrame()

# Process in patient batches
batch_size = 100
for i in range(0, len(patient_ids), batch_size):
    batch = patient_ids.iloc[i:i+batch_size]
    batch_list = tuple(batch['subject_id'])
    
    query = f"""
        SELECT 
            le.subject_id, 
            le.hadm_id, 
            le.itemid, 
            le.charttime, 
            le.valuenum,
            tc.dischtime,
            tc.target
        FROM mimiciv_hosp.labevents le
        JOIN public.temp_cohort tc
          ON le.subject_id = tc.subject_id
         AND le.hadm_id = tc.hadm_id
        WHERE le.charttime BETWEEN (tc.dischtime - INTERVAL '7 days') AND tc.dischtime
        AND le.subject_id IN {batch_list}
    """
    
    chunk = pd.read_sql(query, engine)
    # Process your chunk
    lab_df = pd.concat([lab_df, chunk], ignore_index=True)
# Reset index after concat
lab_df.reset_index(drop=True, inplace=True)

# Pre-processing data

### Aggregating on an `hourly` basis

In [15]:
import polars as pl
patient_data_df = pl.read_parquet("../dataset/raw/lab_events_7_days_prior.parquet")
len(patient_data_df)

10537677

In [13]:
import numpy as np
def assign_time_bin(hours_before_discharge, window_hours=6):
    """Assign records to fixed time bins (e.g., 0-6h, 6-12h).
    Example: For a 6-hour window:
        0.5h → bin 0, 6.1h → bin 6, 23h → bin 18
    """
    return (np.floor(hours_before_discharge / window_hours) * window_hours)

In [ ]:
# Convert timestamps
df = patient_data_df.copy()
df["charttime"] = pd.to_datetime(df["charttime"])
df["dischtime"] = pd.to_datetime(df["dischtime"])

# Calculate hours before discharge
df["hours_before_discharge"] = (
    df["dischtime"] - df["charttime"]
).dt.total_seconds() / 3600

# Filter to 24-hour window (or any max window you need)
df = df[(df["hours_before_discharge"] >= 0) & (df["hours_before_discharge"] <= 168)]

# Assign bins (e.g., 6-hour windows)
df["time_bin"] = assign_time_bin(df["hours_before_discharge"], window_hours=12)
# print(df["time_bin"])

# Create feature IDs (e.g., "itemid_123_last_6h")
df["feature_id"] = (
    "itemid_" + df["itemid"].astype(str) + "_last_" + df["time_bin"].astype(str) + "h"
)

# Numeric features (mean/max)
numeric_features = df.pivot_table(
    index="subject_id",
    columns="feature_id",
    values="valuenum",
    aggfunc="mean", # "max"],  # Customize as needed
)

numeric_features = numeric_features.fillna(-999)  # Fill NaNs with a sentinel value

# Binary features (measurement existence)
df["has_measurement"] = 1
binary_features = df.pivot_table(
    index="subject_id",
    columns="feature_id",
    values="has_measurement",
    aggfunc="max",  # 1 if any measurement exists in the bin
    fill_value=0
)
binary_features.columns = [f"{col}_measured" for col in binary_features.columns]

targets = df[["subject_id", "target"]].drop_duplicates().set_index("subject_id")

numeric_features = numeric_features.join(targets).reset_index()
binary_features = binary_features.join(targets).reset_index()
print(f"Numeric features shape: {numeric_features.shape}")
print(f"Binary features shape: {binary_features.shape}")



Numeric features shape: (2177, 13238)
Binary features shape: (2177, 18554)


In [7]:
numeric_features['subject_id'].duplicated().any()

np.True_

In [6]:
numeric_features.head(3)

,subject_id,itemid_50801_last_164.0h,itemid_50801_last_66.0h,itemid_50802_last_0.0h,itemid_50802_last_10.0h,itemid_50802_last_100.0h,itemid_50802_last_102.0h,itemid_50802_last_104.0h,itemid_50802_last_106.0h,itemid_50802_last_108.0h,...,itemid_53174_last_60.0h,itemid_53174_last_72.0h,itemid_53174_last_74.0h,itemid_53174_last_76.0h,itemid_53174_last_78.0h,itemid_53174_last_84.0h,itemid_53174_last_96.0h,itemid_53174_last_98.0h,itemid_53180_last_44.0h,target
0,10010231,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1
1,10010231,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0
2,10012768,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0


In [7]:
numeric_features['hadm_id'].duplicated().any()

np.False_

In [32]:
# Create a copy and convert timestamps
new_df = patient_data_df.copy()
new_df["charttime"] = pd.to_datetime(new_df["charttime"])
new_df["dischtime"] = pd.to_datetime(new_df["dischtime"])

# Calculate hours before discharge
new_df["hours_before_discharge"] = (new_df["dischtime"] - new_df["charttime"]).dt.total_seconds() / 3600

# Filter to 12-hour window (0 to 12 hours inclusive)
new_df_filtered = new_df[
    (new_df["hours_before_discharge"] >= 0) & (new_df["hours_before_discharge"] <= 12)
].copy()

print(f"Processing {len(new_df_filtered)} records within 12-hour window...")

# Create hourly bins (1-12)
new_df_filtered["hour_bin"] = (np.floor(new_df_filtered["hours_before_discharge"]) + 1).astype(int)
new_df_filtered["hour_bin"] = new_df_filtered["hour_bin"].clip(upper=12)  # Cap at 12

# Create feature_id with hour bin
new_df_filtered["feature_id"] = (
    "itemid_" + 
    new_df_filtered["itemid"].astype(str) + 
    "_last_" + 
    new_df_filtered["hour_bin"].astype(str) + 
    "h"
)

# Pivot numeric features (mean aggregation)
numeric_pivot = new_df_filtered.pivot_table(
    index="hadm_id",
    columns="feature_id",
    values="valuenum",
    aggfunc="mean",
    # fill_value=np.nan,
)
numeric_filled = numeric_pivot.fillna(-999)

# Pivot binary features (existence indicator)
new_df_filtered["has_measurement"] = 1
binary_pivot = new_df_filtered.pivot_table(
    index="hadm_id",
    columns="feature_id",
    values="has_measurement",
    aggfunc="max",  # 1 if any measurement exists
    fill_value=0,
)
binary_pivot.columns = [col + "_measured" for col in binary_pivot.columns]

# combined_features = numeric_filled.join(targets).reset_index()

# Get targets
targets = new_df_filtered[["hadm_id", "target"]].drop_duplicates().set_index("hadm_id")

# Combine features with targets (NO forward/backward fill)
numeric_features = numeric_filled.join(targets).reset_index()
binary_features = binary_pivot.join(targets).reset_index()

print(f"Created numeric features: {numeric_features.shape}")
print(f"Created binary features: {binary_features.shape}")
print(f"Filtered down to {len(new_df_filtered)} rows from {len(new_df)}")
print(f"Number of unique hadm_ids: {new_df_filtered['hadm_id'].nunique()}")

Processing 290346 records within 12-hour window...
Created numeric features: (3076, 1459)
Created binary features: (3099, 2036)
Filtered down to 290346 rows from 3518649
Number of unique hadm_ids: 3099


In [40]:
(np.floor(new_df_filtered["hours_before_discharge"])).astype(int)

368        9
369        9
370        9
371        9
372        9
          ..
3518463    5
3518464    5
3518465    5
3518466    5
3518467    5
Name: hours_before_discharge, Length: 290346, dtype: int64

# Training a Random Forest

In [ ]:
from sklearn.preprocessing import StandardScaler
